In [ ]:
# always run first
from mkndaq.inst.neph import NEPH
from mkndaq.utils.utils import load_config

# path to configuration file
config_file = 'C:/Users/mkn/Documents/git/mkndaq/dist/mkndaq.yml'

# load configuation
cfg = load_config(config_file=config_file)

# # Initialize NEPH
ne300 = NEPH('ne300', cfg, verbosity=2)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# res = ne300.get_id()
# print(res)

In [ ]:
# get setting of data logging target (0: SD card, 1: USB Flash drive)
res = ne300.get_values([2200], verbosity=2)
print(f"DATALOG_MEDIA: {res}")
if res[2200]==1:
    ne300.set_value(parameter_id=2200, value=0)
    print(f"(new) DATALOG_MEDIA: {ne300.get_values([2200], verbosity=2)}")

# get current operation
print(f"CURRENT_OPERATION: {ne300.get_values([4035], verbosity=2)}")

# get current state
print(f"CURRENT_STATE: {ne300.get_values([4036], verbosity=2)}")

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
from datetime import datetime, timedelta
end = datetime.now()
ne300.get_logged_data(end - timedelta(minutes=5), end, raw=True)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
# perform a manual span check for 15' and record 1' data
import time
import schedule

# align start with a 1' timestamp
while int(time.time()) % 60 > 0:
    time.sleep(0.1)

ne300.setup_schedules()

schedule.every(15).seconds.do(ne300.print_ssp_bssp)

# activate span check
t0 = time.time()
ne300.do_span()


while True:
    schedule.run_pending()
    time.sleep(1)
    
    if time.time() - t0 > 900:  # 15 minutes
        ne300.do_ambient()
        ne300._save_and_stage_data()
        break
print(f"Current operation: {ne300.get_current_operation()}")
print("done.")

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
# manually download logged data for the most recent N days
from datetime import datetime, timedelta, timezone
from pathlib import Path

import polars as pl

days = 14
end = datetime.now(timezone.utc)
start = end - timedelta(days=days)
print(f"start: {start}\nend  : {end}")
current = start
while end > current:
    data = ne300.get_logged_data(start=current, end=current + timedelta(minutes=5), verbosity=0)    
    current += timedelta(minutes=5)

print('download completed. Saving to file ...')
df = pl.DataFrame(data)
outfile = Path(cfg['root']).expanduser() / cfg['data'] / cfg['ne300']['data_path'] / 'ne300-manual-download.parquet'
df.write_parquet(outfile)
print(f"data saved to: {outfile}")